In [1]:
name = "CombineSignals"
residual_alphas = [];

In [2]:
using NBInclude
@nbinclude("Alpha.ipynb");

In [10]:
alphas = [
    "UserItemBiases",
    "MatrixFactorization.10",
    "MatrixFactorization.20",
    "MatrixFactorization.40",
    # "MatrixFactorization10",
    # "MatrixFactorization20",
    # "MatrixFactorization40",
    "ItemCF.16",
    "ItemCF.64",
    "ItemCF.256",
    "ItemCF.1024",
];

In [11]:
function evaluate(X, y, β)
    y_pred = X * β
    y_pred = clamp.(y_pred, 1, 10)
    rmse(y, y_pred), mae(y, y_pred), r2(y, y_pred), mse(y, y_pred)
end;

In [12]:
function get_dep(split)
    return get_split(split).rating
end

function get_indep(split, alphas)
    users = get_split(split).user
    X = zeros(length(users), length(alphas))
    for j = 1:length(alphas)
        X[:, j] = get_alpha(alphas[j], split).rating
    end
    X
end;

In [13]:
β = get_indep("validation", alphas) \ get_dep("validation")

11-element Vector{Float64}:
  0.9945339708896017
 -0.40699572280314
  0.14323259490309695
  0.15898102448704596
  0.507809741190879
 -0.00012544686411412387
  0.031225258418709413
  0.17415120657774044
  0.03758317830008589
  0.008227251720132923
  0.47013425425062483

In [14]:
evaluate(get_indep("validation", alphas), get_dep("validation"), β)

(1.1017770402942741, 0.799032479141395, 0.6171301940433394, 1.2139126465196108)

In [15]:
evaluate(get_indep("test", alphas), get_dep("test"), β)

(1.1019003607042563, 0.7991239952435383, 0.6171132456254131, 1.2141844049201704)

In [9]:
write_params(Dict("β" => β, "alphas" => alphas));